In [1]:
!git clone https://github.com/rmitra/DLWorkshop-lab2
%cd DLWorkshop-lab2
!sh getData.sh

fatal: destination path 'DLWorkshop-lab2' already exists and is not an empty directory.
/content/DLWorkshop-lab2
--2020-03-04 10:31:56--  https://www.dropbox.com/s/b20gdo2ywkbx87b/dataset_test_gt.pickle
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b20gdo2ywkbx87b/dataset_test_gt.pickle [following]
--2020-03-04 10:31:56--  https://www.dropbox.com/s/raw/b20gdo2ywkbx87b/dataset_test_gt.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5b5ee6d234524492f0053d0eed.dl.dropboxusercontent.com/cd/0/inline/AzRl_zVTLLaeuPrpt6I6YWpMNLcUd_S7epJlLvK_M3FQlcpX1L6OL-BnmcgWmWkmaCBkcqpfvpAihQbyuICXrWGLAr-1ThoKNwx-zBKouzxyXiWxgRixU5OZc1W2Fs07Nn4/file# [following]
--2020-03-04 10:31:56--  https://uc5b5ee6d234524492f0053d0eed.dl.dro

In [0]:
import os
import numpy as np
import torch
import torch.nn as nn
# from models import Lifter
# from models import Critic
from PoseDataset import PoseDataset
from utils import MPJPE
from utils import project_to_2d
from arguments import parse_args
from tqdm import tqdm
import easydict
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

# Implementing RepNet using PyTorch
## Overview
In this lab, we'll continue from the previous discussions on 3D Human Pose Estimation and try to implement a weakly-supervised method for human pose estimation - Repnet. The method relies on 'reprojection loss' between 2D and 3D poses and is aided by adversarial training.

<div>
    <img src='https://www.cse.iitb.ac.in/~rdabral/dl_workshop/repnet.png' width = 600>
</div>

The method subjects the regressed 3D poses to two main losses:
<ul>
    <li> <b>Reprojection Loss</b>: The poses are projected to 2D using learnt camera parameters. The projected 2D poses are then compared with the input 2D poses. For the purpose of this lab, <b>we'll avoid 'learning' the camera parameters and provide them explicitly</b>.
    <li> <b>Critic Loss</b>: The Critic/Adversarial loss will discriminate between real (gt) and fake (generated) 3D poses. We'll train this with Wasserstein Loss
</ul>

## Model Construction
We begin by defining and constructing the network architecture. The network comprises of two separate networks: 
<ul>
    <li> <b>Lifter</b>: The Lifter network takes as input a batch of 2D poses and converts them to 3D poses. We're calling it 'Lifter' because it is, in essence, <i>lifting</i> the poses from 2D to 3D.
    <li> <b>Critic</b>: The Critic network takes as input a batch of real and fake 3D poses and outputs a 1-d scalar representing whether the poses are real or fake.
</ul>

The Lifter and Critic network architectures look like this:
<div>
    <img src='https://www.cse.iitb.ac.in/~rdabral/dl_workshop/Architecture.png' width = 600>
</div>

We'll start by first writing the model architecture using the nn.Module class we learned about yesterday. You're being provided with the ResNet block pre-implemented. We have also written the constructor for the Lifter.

In [0]:
class ResBlock(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.l1 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.l2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        y = self.l1(x)
        y = self.leaky_relu(y)
        y = self.l2(y)
        z = x + y
        z = self.leaky_relu(z)
        return z

We have also written the constructor for the Lifter model. Your task now is to write the forward of the Lifter model. The input to the Lifter is a 2\*17-dimensional vector and the output is a 3\*17-dimensional vector, just like yesterday.

In [0]:
class Lifter(nn.Module):
    def __init__(self, num_joints, hidden_dim):
        super().__init__()
        self.num_joints = num_joints
        self.hidden_dim = hidden_dim

        # The next blocks are meant for 3D pose regression
        self.pose_reg = nn.Sequential(
                        nn.Linear(2*self.num_joints, self.hidden_dim),
                        nn.LeakyReLU(0.1),
                        ResBlock(self.hidden_dim),
                        ResBlock(self.hidden_dim),
                        ResBlock(self.hidden_dim),
                        nn.Linear(self.hidden_dim, self.hidden_dim),
                        nn.LeakyReLU(0.1),
                        nn.Linear(self.hidden_dim, 3*self.num_joints)
                        )

    def forward(self, input):
        # input: b x n_joints*2
        # poses_3d = 

        return self.pose_reg(input)

In [8]:
Lifter(17,1000)(torch.rand(1,34)).size()

torch.Size([1, 51])

Hope this was easy! Notice that we can use nn.Sequential() to keep our code inside the forward function consize and neat (for the sequential parts, of course). Now, you need to implement the Critic network on your own. The input to the Critic will be a 3D pose: 3*17 dimensional vector and output will be a scalar (1-d).

In [0]:
class Critic(nn.Module):
    def __init__(self, num_joints, hidden_dim):
        super().__init__()
        self.num_joints = num_joints
        self.hidden_dim = hidden_dim

        self.l1 = nn.Linear(3*num_joints, self.hidden_dim)
        self.act=nn.LeakyReLU(0.1)
        self.res=ResBlock(self.hidden_dim)
        self.l2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.l3 = nn.Linear(self.hidden_dim, 1)
        self.sig=nn.Sigmoid()
        # Initialize your network layers here

    def forward(self, input):
        # Forward propagate the input through your network layers here
        out = self.act(self.l1(input))
        out = self.l3(self.act(self.res(self.l2(out))))
        return self.sig(out)


In [17]:
Critic(17,1000)(torch.rand(1,51)).size()

torch.Size([1, 1])

Now that we're done defining the models, let's jump into some rountine stuffs. We'll initialize the two models and push them to cuda. We'll also initialize two optimizers, one for each model. Why do we need different optimizers for the two? Because GANs!

In [0]:
cfg = easydict.EasyDict({'batch_size': 32,
        'epochs': 10,
        'learning_rate': 0.00005,})

lifter = Lifter(num_joints=17, hidden_dim=1000)
lifter.cuda()
lifter_optimizer = torch.optim.Adam(lifter.parameters(),
                                        lr=cfg.learning_rate,
                                        betas=(0.5, 0.9)
                                        )

# Initializing the adversarial/critic module
critic = Critic(num_joints=17, hidden_dim=100)
critic.cuda()

# We think Adam optimizer should work well for our critic. But you are
# encouraged to explore other ones, say RMSprop, or SGD. Google!
critic_optimizer = torch.optim.Adam(critic.parameters(),
                                        lr=cfg.learning_rate,
                                        betas=(0.5, 0.9)
                                        )


The next piece of the jigsaw is the data. This is complex; probably, the clumsiest part of any deep learning code! We have implemented a DataSet class for you. <b>Ask questions if you're interested. Or, we can treat it as a blackbox.</b> The dataset object is given to PyTorch's dataloader class whose sole job is to efficiently churn out nicely arranged batches of data for you. Let's try to see what the data looks like.

In [19]:
# Data Loaders
train_dataset = PoseDataset(cfg, split='train')
train_loader = torch.utils.data.DataLoader(
                                train_dataset,
                                batch_size = cfg.batch_size,
                                shuffle = True,
                                num_workers = 4
                                )

# train_loader is an iterator. We can use enumerate to iterate over it:
for i, (pose_2d, pose_3d_real, cams, roots) in enumerate(train_loader):
  # This is the input to the Lifter Network
  print(pose_2d.shape)      
  # These are some random 3D poses. They'll be used to train the Critic.
  print(pose_3d_real.shape)
  # These are the camera parameters. They'll be used to project the predicted
  # 3D poses back to 2D image coordinates. 
  print(cams.shape)
  # Ignore this. These are also used for reprojection.
  print(roots.shape)
  break

test_dataset = PoseDataset(cfg, split='test')
test_loader = torch.utils.data.DataLoader(
                                test_dataset,
                                batch_size = cfg.batch_size,
                                shuffle = True,
                                num_workers = 4
                                )


 Loaded 312188 samples
torch.Size([32, 34])
torch.Size([32, 51])
torch.Size([32, 9])
torch.Size([32, 1, 3])

 Loaded 108772 samples


We're done with the models, optimizers and the data. The last things we need are the loss functions. We'll discuss their use in detail later.

In [0]:
rep_loss = nn.MSELoss().cuda()
crit_loss = nn.BCELoss().cuda()


Let's define the training loop now. For the purposes of this code, we call the train() function for every epoch. The dataloader iterates over all the batches of the dataset withing this function. The training iteration will follow the following algorithm:
<ul>
  <li> Forward prop the 2D input through the Lifter to get the 3D pose predictions.
  <li> Train the Critic:
        <ul>
          <li> Forward prop the 3D predictions (fake) through the Critic
          <li> Forward prop the Real 3D poses through the Critic. Note that these 'real' 3D poses have no relation with the predicted (or fake) 3D poses. They've simply been sampled from the training dataset.
          <li> Compute and backpropagate the Wasserstein-loss on the real and fake outputs. Note, that <b>we do not want to backprop the Lifter model with this loss</b>. To avoid that from happening, use detach().
        </ul>
  <li> Train the Lifter:
        <ul>
          <li> Forward prop the 3D predictions through the Critic.
          <li> Compute the Wasserstein-loss on the predicted poses. This time, the loss needs to be backpropagated to the Lifter model as well. The higher this loss, the better our Lifter! Why?
          <li> Use the camera parameters provided by the data-loader to project the 3D pose predictions into 2D poses. 
          <li> Compute the reprojection loss. It is the MSE loss between the input 2D poses (to the Lifter) and the projected 2D poses (from the Lifter).
          <li> Add the critic and reprojection losses and backpropagate them through the lifter module. Update the Lifter weights.

In [0]:
def train(epoch, lifter, critic, lifter_optimizer, critic_optimizer,
                                rep_loss, crit_loss, train_loader):

    # train
    rep_loss_total, lifter_critic_loss_total = 0, 0
    discriminator_loss_total = 0
    train_loader.dataset.init_epoch()

    for i, (input_2d, real_poses_3d, cameras, root) in enumerate(tqdm(train_loader, ascii=True)):
        # Constant Tensors to be used as labels for losses
        valid = torch.ones((real_poses_3d.size(0), 1)).float().cuda()
        fake = torch.zeros((input_2d.size(0), 1)).float().cuda()

        input_2d = input_2d.cuda()
        real_poses_3d = real_poses_3d.cuda()
        cameras = cameras.cuda()
        root = root.cuda()
        poses_3d = lifter(input_2d)

        # ---------------- Training the Critic ------------------

        real_validity = critic(real_poses_3d)
        fake_validity = critic(poses_3d.detach())
        # Wasserstein-loss for the Critic

        #when we pass in real image we want outputs to be close to 1 i.e as high as possible and vice versa for fake *****
        # so for real validity we want to increase the output and for fake decrease
        # so max real_out + (-fake_out)
        # For torch's backward: 
        critic_loss = -torch.mean(real_validity) + torch.mean(fake_validity) 
        # this is wassertein so we just act in a raw way on these outputs
        # Generally we would do this for the cross-entropy output:
        # Traditional GAN loss
        # critic_loss = 0.5 * (crit_loss(real_validity, valid) + crit_loss(fake_validity, fake))
        # this includes the ce loss with all 1's and 0's labels

        
        # Flush the existing gradients and backpropagate the loss
        critic_optimizer.zero_grad()
        critic_loss.backward()
        critic_optimizer.step()

        #  ------------- Training the Generator -----------------

        # Wasserstein Loss for the Lifter
        lifter_critic_loss = -torch.mean(critic(poses_3d))
        # Traditional GAN loss
        # lifter_critic_loss = nn.crit_loss(critic(poses_3d), valid)

        # Reprojection loss
        poses_rep = project_to_2d(poses_3d, cameras, root)
        reprojection_loss = rep_loss(poses_rep, input_2d)

        # Combined loss
        lifter_loss = reprojection_loss + lifter_critic_loss

        # Flush the existing gradients and backpropagate the loss
        lifter_optimizer.zero_grad()
        lifter_loss.backward()
        lifter_optimizer.step()



        # --------------- Book Keeping --------------------------
        rep_loss_total += reprojection_loss.cpu().item()
        lifter_critic_loss_total += lifter_critic_loss.cpu().item()
        discriminator_loss_total += critic_loss.cpu().item()

    print(f'epoch: {epoch}, reprojection_loss: {rep_loss_total / (i+1) :.6f}, \
            lifter_critic_loss = {lifter_critic_loss_total / (i+1) :.4f}, \
            discriminator_critic_loss = {discriminator_loss_total / (i+1) :.6f}')


We're implementing a function to validate the performance of our Lifter model. This will compute the Mean Per Joint Position Error at the end of every epoch.

In [0]:
def val(epoch, lifter, test_loader):
    # Setting the model's mode to eval mode
    lifter.eval()

    mpjpe = 0
    for i, (input_2d, gt_3d, cameras) in enumerate(test_loader):
        input_2d = input_2d.cuda()
        gt_3d = gt_3d.cuda()
        cameras = cameras.cuda()

        pred_3d = lifter(input_2d)

        # Calculate the Mean Per Joint Position Error
        mpjpe += MPJPE(pred_3d, gt_3d)

    print(f"The MPJPE for epoch {epoch} is {mpjpe/i}")

    # Resetting the model back to training mode
    lifter.train()


Let's train the entire thing for 10 epochs and see what happens.


In [0]:
# Enter the training loop
for epoch in range(cfg.epochs):
    train(epoch, lifter, critic, lifter_optimizer, critic_optimizer,
                            rep_loss, crit_loss, train_loader)

    if (epoch+1) % 1 == 0:
        val(epoch, lifter, test_loader)


100%|##########| 9756/9756 [03:36<00:00, 45.00it/s]

epoch: 0, reprojection_loss: 0.012230,             lifter_critic_loss = -0.0479,             discriminator_critic_loss = 0.006636



  0%|          | 0/9756 [00:00<?, ?it/s]

The MPJPE for epoch 0 is 128.42086791992188


100%|##########| 9756/9756 [03:41<00:00, 44.03it/s]

...

epoch: 5, reprojection_loss: 0.000181,             lifter_critic_loss = -0.0047,             discriminator_critic_loss = -0.000170



  0%|          | 0/9756 [00:00<?, ?it/s]

The MPJPE for epoch 5 is 123.29306030273438


100%|##########| 9756/9756 [03:45<00:00, 43.28it/s]

epoch: 6, reprojection_loss: 0.000321,             lifter_critic_loss = -0.0135,             discriminator_critic_loss = -0.000572


Voila! You just implemented a CVPR paper :-)